In [5]:
# Preconditions

import os
import json
from PIL import Image
import numpy as np
import keras.api._v2.keras as keras
from keras.applications import VGG16, ResNet50
from keras.callbacks import EarlyStopping, TensorBoard
from keras.callbacks import ReduceLROnPlateau
from keras import regularizers
import tensorflow as tf
import random

import helper as hp

os.environ["TF_NUM_INTEROP_THREADS"] = "1"  # Number of inter-op threads
os.environ["TF_NUM_INTRAOP_THREADS"] = "4"  # Number of intra-op threads

MODEL_NAME = "v1_E50_PAT4_LR_001s_1000" + ".keras"
MODEL_PATH = os.path.join("..", "tmp", "models")

RESSOURCES_PATH = os.path.join("..", "tmp", "train", "ressources", "bundle")
TRAIN_DATA_FOLDER = [
    # "1f7534fa-6ee2-4e6a-a921-2a635a5fe917", 
    # "9ef5620b-c769-49ba-b083-7cbc25fe7ec6",
    # "56dd460c-200d-4248-b6d5-b61bd0681fd7",
    # "81370b3a-0747-44f7-bdd9-08279027b99a",
    # "470833f8-ddaf-4aca-8a65-e30f80504a8a",
    # "32831052-7914-4220-bde4-c970c9c6c404",
    # "82612320-658a-495f-9d10-e54b35471628",
    # "a04111fd-ac8f-44b4-b1af-fe8f67252098",
    # "b8768080-7bdd-43c1-ab11-b940b74b07ef",
    # "efc4db14-5e6b-4c14-8543-997997d55476"
    # "e80ee18d-5caf-420c-9c89-3062df2a30fb",
    # "d164f634-b0af-46a6-a0f6-c85c0b462645",
    # "ce40130e-f719-4921-ba4c-1529aa7158ba",
    # "623bf116-6dcb-4293-860f-7c777f132c1a",
    # "6dcf2542-014c-4e55-b8ff-2811c3f81fa2"
    # "6b6f2d7c-5fcb-4912-a0f9-2f0ea405c766",
    # "80b3b3b8-7521-415a-a584-24d2b94faa19",
    # "2901923e-69a3-450a-8916-790fb6ab0ae6",
    # "37470658-b16b-4878-9a55-29ffe7815a9a",
    # "d67ed669-e26b-494c-b594-b01756e5a03f"

    # "05c1d782-71b2-4e21-a857-97af015c2fc4",
    # "3c9332f3-7f01-4881-8f34-10f7ad182ace",
    # "10615d94-22fd-4dad-915b-bb3c74458200",
    # "c52325af-e217-4b87-8e71-f24d9b5dbc44",
    # "3d442115-ff43-4c76-9051-14df9ae62e37"
    # "508ceb38-8d56-46cf-8537-c4c134758bff"
    # "9e06cd91-0a34-4272-9afd-135424b077ce"

    # "8db6808f-9497-4fb7-b541-a83c79c63c9d",
    # "59db0623-7f36-48cf-a8b4-5cc63cc56746",
    # "4ff3606d-d18e-42e5-876d-8518a663b868",
    # "f84e6a0e-3497-4376-887b-6dbf6b443893",
    # "c2b72ca1-f566-49ba-a298-53a5f6b37c7e"

    "bfee6c3a-7fd7-42c5-b933-9cd8224623eb"
]

# Image properties
IMAGE_HEIGHT_PX = 120
IMAGE_WIDTH_PX = 160
CROP_HEIGHT_PX = 5
CROP_WIDTH_PX = 20

NUM_CHANNELS = 3
NUM_CLASSES = 4
NUM_POSITIONS = 8

color_mapping = { 'red': 0, 'yellow': 1, 'blue': 2, '': 3 }
label_mapping = { 0: 'red', 1: 'yellow', 2: 'blue', 3: ''}

NORMALIZE_VALUE = 255

def map_labels_to_nummeric(label):
    mapped_label = []

    for pos in label.values():
        mapped_label.append(color_mapping[pos])

    return mapped_label

# Normalize the images so that all values are between 0 and 1
def normalize_images(images):
    return images / NORMALIZE_VALUE


# Data loading
IN_DEBUG_MODE = False
IMAGE_FOLDER = "Images"
LABELS_FOLDER = "Labels"
JSON_NAME = "scene_results.json"

def get_data(stage):
    labels = []
    images = []

    for train_folder in TRAIN_DATA_FOLDER:

        scene_results_path = os.path.join(RESSOURCES_PATH, train_folder, stage, JSON_NAME)

        if IN_DEBUG_MODE:
            print("CURRENT STAGE: " + stage + "\n")
            print("READING SCENE RESULTS AT: " + scene_results_path)

        with open(scene_results_path, 'r') as file:
            scene_results = json.load(file)

        for result in scene_results:

            img = [];

            image_not_found = False

            if len(result["imagePaths"]) != 2:
                continue

            for img_path in result["imagePaths"]:

                if IN_DEBUG_MODE:
                    print("READING IMAGE AT: " + img_path)

                try:
                    i = Image.open(os.path.join(RESSOURCES_PATH, train_folder, img_path))
                except:
                    image_not_found = True
                    continue


                # Scale down image (resize)
                i = i.resize((IMAGE_WIDTH_PX, IMAGE_HEIGHT_PX))

                # Channel order of Pillow is different than OpenCV
                i = np.array(i)
                i = hp.Preprocess.convert_to_BGR(i)

                # i = hp.Video.translate_image(i)

                # Crop the image
                i = i[0:115, 10:150]

                # i = hp.Augmentation.black_spots(i, 10)

                i = hp.Preprocess.start(i)

                img.append(i)

            if image_not_found == False:
                images.append(img)
                try:
                    # np.array(images) # is only necessary to check if the data is homogenous
                    labels.append(result["positions"])
                except:
                    print(f"Images shape got inhomogenous at: ${result['imagePaths']}")
                    images.pop()

        if IN_DEBUG_MODE:
            print("\n\n")
    


    return [np.array(images), np.array(labels)]

In [5]:
# Base-Model generation
LOSS_FUNCTION = 'categorical_crossentropy'

input_branch_1 = keras.layers.Input(shape=(IMAGE_HEIGHT_PX - CROP_HEIGHT_PX, IMAGE_WIDTH_PX - CROP_WIDTH_PX, 3))
input_branch_2 = keras.layers.Input(shape=(IMAGE_HEIGHT_PX - CROP_HEIGHT_PX, IMAGE_WIDTH_PX - CROP_WIDTH_PX, 3))

# Shared convolutional layers for image processing
convolutional_layers = [
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten()
]

# Process first image
x1 = input_branch_1
for layer in convolutional_layers:
    x1 = layer(x1)

# Process second image
x2 = input_branch_2
for layer in convolutional_layers:
    x2 = layer(x2)

x = keras.layers.Concatenate(axis=-1)([x1, x2])
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(NUM_POSITIONS * NUM_CLASSES, activation='softmax')(x)  # Output layer with 8 * 4 units

output = keras.layers.Reshape((NUM_POSITIONS, NUM_CLASSES))(x)

# Build the model with the two input branches and the output layer
base_model = keras.models.Model(inputs=[input_branch_1, input_branch_2], outputs=output)

optimizer = keras.optimizers.legacy.Adam(learning_rate=0.001)

base_model.compile(optimizer=optimizer, loss=LOSS_FUNCTION, metrics=['accuracy', 'mean_squared_error'])

In [6]:
# Base model generation v2
LOSS_FUNCTION = 'categorical_crossentropy'

base_model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_HEIGHT_PX - CROP_HEIGHT_PX, IMAGE_WIDTH_PX - CROP_WIDTH_PX, 3)),
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_HEIGHT_PX - CROP_HEIGHT_PX, IMAGE_WIDTH_PX - CROP_WIDTH_PX, 3), kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),

    keras.layers.Dropout(0.2),
    keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(128, activation='relu')
])

input_image_1 = keras.layers.Input(shape=(IMAGE_HEIGHT_PX - CROP_HEIGHT_PX, IMAGE_WIDTH_PX - CROP_WIDTH_PX, 3), name='input_image_1')
input_image_2 = keras.layers.Input(shape=(IMAGE_HEIGHT_PX - CROP_HEIGHT_PX, IMAGE_WIDTH_PX - CROP_WIDTH_PX, 3), name='input_image_2')

encoded_left = base_model(input_image_1)
encoded_right = base_model(input_image_2)

concatenated = keras.layers.Concatenate(axis=-1)([encoded_left, encoded_right])

dense_layer = keras.layers.Dense(NUM_POSITIONS * NUM_CLASSES, activation='softmax')(concatenated)
reshaped_output = keras.layers.Reshape((NUM_POSITIONS, NUM_CLASSES))(dense_layer)

base_model =  keras.models.Model(inputs=[input_image_1, input_image_2], outputs=reshaped_output)

optimizer = keras.optimizers.legacy.Adam(learning_rate=0.001)

base_model.compile(optimizer=optimizer, loss=LOSS_FUNCTION, metrics=['accuracy', 'mean_squared_error'])

In [33]:

# Base model with VGG16

# Input layers for the two images
input_branch_1 = keras.layers.Input(shape=(IMAGE_HEIGHT_PX - CROP_HEIGHT_PX, IMAGE_WIDTH_PX - CROP_WIDTH_PX, 3))
input_branch_2 = keras.layers.Input(shape=(IMAGE_HEIGHT_PX - CROP_HEIGHT_PX, IMAGE_WIDTH_PX - CROP_WIDTH_PX, 3))

# Load the pre-trained VGG16 model (without the top layer)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_HEIGHT_PX - CROP_HEIGHT_PX, IMAGE_WIDTH_PX - CROP_WIDTH_PX, 3))

# Freeze the pre-trained layers (optional)
for layer in base_model.layers:
  layer.trainable = False

# Extract features from both images using the VGG16 base model
x1 = base_model(input_branch_1)
x2 = base_model(input_branch_2)

# Concatenate the extracted features
x = keras.layers.Concatenate(axis=-1)([x1, x2])

# Add custom layers for your specific task
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
# x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(NUM_POSITIONS * NUM_CLASSES, activation='softmax')(x)  # Output layer with 8 * 4 units

# Final output layer for multi-class classification (adjust based on your problem)
output = keras.layers.Reshape((NUM_POSITIONS, NUM_CLASSES))(x)

# Create the final model with two inputs and one output
base_model = keras.models.Model(inputs=[input_branch_1, input_branch_2], outputs=output)

# Compile the model (adjust optimizer and loss function as needed)
base_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'mean_squared_error'])

In [2]:

# Base model with RESNET

# Input layers for the two images
input_branch_1 = keras.layers.Input(shape=(IMAGE_HEIGHT_PX - CROP_HEIGHT_PX, IMAGE_WIDTH_PX - CROP_WIDTH_PX, 3))
input_branch_2 = keras.layers.Input(shape=(IMAGE_HEIGHT_PX - CROP_HEIGHT_PX, IMAGE_WIDTH_PX - CROP_WIDTH_PX, 3))

# Load the pre-trained ResNET50 model (without the top layer)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMAGE_HEIGHT_PX - CROP_HEIGHT_PX, IMAGE_WIDTH_PX - CROP_WIDTH_PX, 3))

# Freeze the pre-trained layers (optional)
for layer in base_model.layers:
  layer.trainable = False

# Extract features from both images using the ResNet50 base model
x1 = base_model(input_branch_1)
x2 = base_model(input_branch_2)

# Concatenate the extracted features
x = keras.layers.Concatenate(axis=-1)([x1, x2])

# Add custom layers for your specific task
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
# x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(NUM_POSITIONS * NUM_CLASSES, activation='softmax')(x)  # Output layer with 8 * 4 units

# Final output layer for multi-class classification (adjust based on your problem)
output = keras.layers.Reshape((NUM_POSITIONS, NUM_CLASSES))(x)

# Create the final model with two inputs and one output
base_model = keras.models.Model(inputs=[input_branch_1, input_branch_2], outputs=output)

# Compile the model (adjust optimizer and loss function as needed)
base_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'mean_squared_error'])

In [32]:
# Re-train existing model
base_model = keras.models.load_model(os.path.join(MODEL_PATH, "v5_no_base_70000.keras"))

In [7]:
train_data = get_data("Train")
verify_data = get_data("Verify")

In [8]:
# Train model
early_stopping = EarlyStopping(monitor='val_loss', patience=4, min_delta=0.01, restore_best_weights=True)
learning_rate_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_delta=0.01)
tensorboard_callback = TensorBoard(log_dir='./logs')

IN_DEBUG_MODE = False
EPOCS = 50

def fit_model(model, train_data, verify_data):
    train_images = normalize_images(np.array(train_data[0]))
    numberic_train_labels = np.array([map_labels_to_nummeric(label) for label in train_data[1]])
    train_labels = keras.utils.to_categorical(numberic_train_labels, num_classes=NUM_CLASSES)

    verify_images = normalize_images(np.array((verify_data[0])))
    numberic_verify_labels = np.array([map_labels_to_nummeric(label) for label in verify_data[1]])
    verify_labels = keras.utils.to_categorical(numberic_verify_labels, num_classes=NUM_CLASSES)

    if IN_DEBUG_MODE: 
        print("----- SHAPES ------\n")
        print(f"Train labels shape: {train_labels.shape}")
        print(f"Train images shape: {train_images.shape}")

        print(f"Verify labels shape: {verify_labels.shape}")
        print(f"Verify images shape: {verify_images.shape}\n\n")

    model.fit(
        [train_images[:, 0], train_images[:, 1]], train_labels, 
        epochs=EPOCS, 
        validation_data=([verify_images[:, 0], verify_images[:, 1]], verify_labels), verbose=1,
        callbacks=[early_stopping, learning_rate_scheduler, tensorboard_callback])
    
    return model

trained_model = fit_model(base_model, train_data, verify_data)
trained_model.save(os.path.join(MODEL_PATH, MODEL_NAME))

Epoch 1/50
22/22 [==============================] - 20s 906ms/step - loss: 2.6122 - accuracy: 0.3923 - mean_squared_error: 0.2332 - val_loss: 1.8739 - val_accuracy: 0.4200 - val_mean_squared_error: 0.2318 - lr: 0.0010
Epoch 2/50
22/22 [==============================] - 20s 912ms/step - loss: 1.6332 - accuracy: 0.4448 - mean_squared_error: 0.2300 - val_loss: 1.5157 - val_accuracy: 0.4406 - val_mean_squared_error: 0.2297 - lr: 0.0010
Epoch 3/50
22/22 [==============================] - 20s 925ms/step - loss: 1.3867 - accuracy: 0.4950 - mean_squared_error: 0.2271 - val_loss: 1.3582 - val_accuracy: 0.4988 - val_mean_squared_error: 0.2280 - lr: 0.0010
Epoch 4/50
22/22 [==============================] - 20s 932ms/step - loss: 1.2340 - accuracy: 0.5539 - mean_squared_error: 0.2236 - val_loss: 1.2331 - val_accuracy: 0.5481 - val_mean_squared_error: 0.2229 - lr: 0.0010
Epoch 5/50
22/22 [==============================] - 21s 965ms/step - loss: 1.1022 - accuracy: 0.6391 - mean_squared_error: 0.218

In [6]:
# Save trained model 
trained_model.save(os.path.join(MODEL_PATH, MODEL_NAME))

In [23]:
# Load model
model = keras.models.load_model(os.path.join(MODEL_PATH, MODEL_NAME))

In [24]:
test_data = get_data("Test")

In [25]:
# Test model
test_labels = np.array([map_labels_to_nummeric(label) for label in test_data[1]])

if IN_DEBUG_MODE:
    model.summary()

test_images = normalize_images(np.array((test_data[0])))
predictions = model.predict([test_images[:, 0], test_images[:, 1]])

label_index = random.randint(0, len(test_labels)-1)

print("\n\n")
print(f"------ PREDICTION: Index {label_index + 1} --------\n")
predicted_nummeric = np.argmax(predictions, axis=-1)
predicted_readable = np.vectorize(label_mapping.get)(predicted_nummeric)
actual_readable = np.vectorize(label_mapping.get)(test_labels)


print("NUMMERIC: \n")
print(predicted_nummeric[label_index])
print("READABLE: \n")
print(predicted_readable[label_index])
print("\n\n")

print(f"------ ACTUAL: Index {label_index + 1} ------ \n")
print("NUMMERIC: \n")
print(test_labels[label_index])
print("READABLE: \n")
print(actual_readable[label_index])

32/32 [==============================] - 2s 74ms/step



------ PREDICTION: Index 394 --------

NUMMERIC: 

[3 1 3 2 3 2 3 0]
READABLE: 

['' 'yellow' '' 'blue' '' 'blue' '' 'red']



------ ACTUAL: Index 394 ------ 

NUMMERIC: 

[3 1 1 2 3 2 2 0]
READABLE: 

['' 'yellow' 'yellow' 'blue' '' 'blue' 'blue' 'red']
